In [4]:
FOLDER_PATH = "./hwp_test_3files"

EXCLUDE_DIRS = {
    ".git", "__pycache__", "node_modules", ".ipynb_checkpoints",
    ".venv", "venv", ".mypy_cache", ".ruff_cache", ".pytest_cache",
    "build", "dist"
}
EXCLUDE_FILE_PATTERNS = ["*.pyc", "*.pyo", "*.o", "*.so", "*.DS_Store", "Thumbs.db"]



In [5]:
import os
from datetime import datetime
from pathlib import Path

root = Path(FOLDER_PATH).resolve()
if not root.exists():
    raise FileNotFoundError(f"폴더를 찾을 수 없습니다: {root}")

BASE_NAME = root.name
TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
OUTPUT_ZIP = f"{BASE_NAME}_{TIMESTAMP}.zip"

print(f"[확인] 대상 폴더: {root}")
print(f"[확인] 출력 ZIP : {OUTPUT_ZIP}")

[확인] 대상 폴더: /home/spai0316/projectmission2/models/hwp_test_3files
[확인] 출력 ZIP : hwp_test_3files_20250916_001349.zip


In [6]:
import os, fnmatch, zipfile
from pathlib import Path

# tqdm이 없으면 무진행 표시로 대체
try:
    from tqdm import tqdm
except Exception:
    def tqdm(x, **k): return x

def should_exclude(path: Path) -> bool:
    # 경로 안에 제외 디렉터리가 포함되면 제외
    for part in path.parts:
        if part in EXCLUDE_DIRS:
            return True
    # 파일 패턴 제외
    for pat in EXCLUDE_FILE_PATTERNS:
        if fnmatch.fnmatch(path.name, pat):
            return True
    return False

# 압축 대상 파일 수집
all_files = []
for p, dirs, files in os.walk(root):
    # 디렉터리 단계에서 제외 디렉터리 가지치기(성능 ↑)
    dirs[:] = [d for d in dirs if d not in EXCLUDE_DIRS]
    for f in files:
        fp = Path(p) / f
        if not should_exclude(fp):
            all_files.append(fp)

print(f"[정보] 압축 대상 파일 수: {len(all_files)}")

# ZIP64(2GB+) 지원, Deflate 압축
with zipfile.ZipFile(OUTPUT_ZIP, "w", compression=zipfile.ZIP_DEFLATED, allowZip64=True) as zf:
    for fp in tqdm(all_files, desc="Zipping", unit="files"):
        arcname = fp.relative_to(root)  # 폴더 상대경로로 보존
        zf.write(fp, arcname)

print(f"[완료] 생성된 ZIP: {OUTPUT_ZIP}")

[정보] 압축 대상 파일 수: 5


Zipping: 100%|██████████| 5/5 [00:15<00:00,  3.05s/files]

[완료] 생성된 ZIP: hwp_test_3files_20250916_001349.zip


In [7]:
import os, hashlib

def sha256sum(path, block_size=1024*1024):
    h = hashlib.sha256()
    with open(path, "rb") as f:
        while True:
            b = f.read(block_size)
            if not b: break
            h.update(b)
    return h.hexdigest()

def human_bytes(n: int) -> str:
    units = ["B","KB","MB","GB","TB","PB"]
    size = float(n)
    for u in units:
        if size < 1024:
            return f"{size:.2f} {u}"
        size /= 1024
    return f"{size:.2f} EB"

size = os.path.getsize(OUTPUT_ZIP)
print(f"[ZIP] {OUTPUT_ZIP}")
print(f"[크기] {human_bytes(size)}")
print(f"[SHA256] {sha256sum(OUTPUT_ZIP)}")

# (선택) ZIP 내부 샘플 확인
import zipfile
with zipfile.ZipFile(OUTPUT_ZIP, "r") as zf:
    names = zf.namelist()[:15]
print("[샘플 15개 항목]")
for n in names:
    print(" └─", n)

[ZIP] hwp_test_3files_20250916_001349.zip
[크기] 223.65 MB
[SHA256] 7ebd3db875f07aa6cdb9de30e203f8c91c62a7513270455449377fe3273846c2
[샘플 15개 항목]
 └─ pipeline_config.json
 └─ embeddings.npy
 └─ chunks.jsonl
 └─ vector_store/faiss_index.bin
 └─ vector_store/metadata.json


In [8]:
from IPython.display import FileLink, display

# Jupyter(Lab) / VSCode 노트북: 클릭해서 다운로드
display(FileLink(OUTPUT_ZIP, result_html_prefix="클릭하여 다운로드: "))

# Google Colab 환경이면 자동 다운로드 시도
try:
    from google.colab import files as colab_files
    colab_files.download(OUTPUT_ZIP)
except Exception as e:
    # Colab이 아니거나 제한으로 실패한 경우
    print("Colab 자동 다운로드가 아니거나 제한으로 인해 실패했습니다.")
    print("위의 링크를 클릭하여 다운로드하세요.")

/home/spai0316/projectmission2/models/hwp_test_3files_20250916_001349.zip

Colab 자동 다운로드가 아니거나 제한으로 인해 실패했습니다.
위의 링크를 클릭하여 다운로드하세요.
